In [1]:
import os
%pwd

'c:\\Users\\MEDHA TRUST\\Documents\\Govardhan\\ML\\github\\CustomerChurn\\CustomerChurn\\research'

In [2]:
## command should be in CustomerChurn directory
os.chdir('../')
%pwd

'c:\\Users\\MEDHA TRUST\\Documents\\Govardhan\\ML\\github\\CustomerChurn\\CustomerChurn'

In [22]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_path: Path
    C: float
    max_iter: int
    penalty: str
    solver: str
    target_column: str


In [31]:
from ChurnPrediction.constants import *
from ChurnPrediction.utils.common import read_yaml, create_directory

class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directory([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.LogisticRegression
        schema = self.schema.TARGET_COLUMN

        create_directory([config.root_dir])
        
        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_path = config.model_path,
            C = params.C,
            max_iter = params.max_iter,
            penalty = params.penalty,
            solver = params.solver,
            target_column = list(schema.keys())[0]
        )

        return model_trainer_config


In [37]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from ChurnPrediction import logger
import joblib
import os

In [38]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        train_X = train_data.drop([self.config.target_column], axis = 1)
        train_y = np.ravel(train_data[[self.config.target_column]])
        test_X = test_data.drop([self.config.target_column], axis = 1)
        test_y = np.ravel(test_data[[self.config.target_column]])

        model = LogisticRegression(
            C = self.config.C,
            max_iter = self.config.max_iter, 
            penalty = self.config.penalty, 
            solver = self.config.solver
        )

        model.fit(train_X, train_y)

        joblib.dump(model, Path(self.config.model_path))


        

In [39]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config = model_trainer_config)
    model_trainer.train()

except Exception as e:
    raise Exception

[2023-10-16 00:50:41,150: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-10-16 00:50:41,160: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-16 00:50:41,168: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-10-16 00:50:41,168: INFO: common: directory: artifacts created successfully]
[2023-10-16 00:50:41,176: INFO: common: directory: artifacts/model created successfully]
